In [8]:
import sys
sys.path.append('../')
from data import create_loader, create_dataset
from timm.models import create_model
from models import vision_transformer

import torch

In [16]:
model_path = '/data/hjy/SSF/vit_base_patch16_224_in21k/vtab/caltech101/baseline-92.88-173.pth.tar'

class ARGS:
    def __init__(self):
        self.model = 'vit_base_patch16_224_in21k'
        self.pretrained = True
        self.num_classes = 102
        self.drop = 0
        self.drop_connect = None
        self.drop_path = 0
        self.drop_block = None
        self.gp = None
        self.bn_momentum = None
        self.bn_eps = None
        self.torchscript = False
        self.initial_checkpoint = None
        self.tuning_mode = 'ssf'

        self.dataset = 'caltech101'
        self.data_dir = '/data/datasets/VTAB/VTAB/vtab-1k/caltech101'
        self.val_split = 'validation'
        self.class_map = ''
        self.dataset_download = False
        self.batch_size = 128

args = ARGS()

In [10]:
model = create_model(
        args.model,
        pretrained=args.pretrained,
        num_classes=args.num_classes,
        drop_rate=args.drop,
        drop_connect_rate=args.drop_connect,  # DEPRECATED, use drop_path
        drop_path_rate=args.drop_path,
        drop_block_rate=args.drop_block,
        global_pool=args.gp,
        bn_momentum=args.bn_momentum,
        bn_eps=args.bn_eps,
        scriptable=args.torchscript,
        checkpoint_path=args.initial_checkpoint,
        tuning_mode=args.tuning_mode)

model.cuda()
checkpoint = torch.load(model_path)
model_state_dict = checkpoint['state_dict']
model.load_state_dict(model_state_dict, strict=False)

<All keys matched successfully>

In [17]:
dataset_eval = create_dataset(
        args.dataset, root=args.data_dir, split=args.val_split, is_training=False,
        class_map=args.class_map,
        download=args.dataset_download,
        batch_size=args.batch_size)